  
Supplementary code for the MPC Monetary Experiment paper
============
-----

## **Calibration on 1945 - now and post Covid debt  simulations**

Here is an interactive Notebook to load and play directly with the Matlab code used in the MPC Monetary Experiment paper (available here), adapted from the Bogomolnaia and Moulin cake eating algorithm paper (2001). Please allow Binder to be loaded (<1 min), then you can look at the code, change parameters and run cell by cell as in any Jupyter Notebook (Maj+Enter to run each cell once, or use menu shortcut to run all cells under Cell > "Run All").  

**The model used here is detailed and presented more in details in the Generic "Cake-Eating" based algorithms discussion notebook (https://mybinder.org/v2/gh/NicolasXYZ/MPC_experiment/master?filepath=index.ipynb). We only remind the main features of the model here, before focusing on the calibration and on simulating world debts, 1945-now, and several post Covid-19 scenarii**

### Bogomolnaia and Moulin's cake eating algorithm (2001), adapted to macro

- Time is discrete, in rounds.
- At the start of each round each country generates one "cake" (debt) 
- Each country simultaneously eats each others' cakes (debt) at different speed and following different rankings (so when a piece of cake is finished the countries that were eating eat move to the item below in their rankings)
- Each country stores what he ate in his reserves - the endowments
- If at the end of a round a country's cake has not been completed absorbed (by itself and/or by other countries) it suffers a devaluation - i.e. every the value of pieces of cake from this country store in others' endowments is decreased, and its interest rate rises.
- Countries' ranking preferences are function of their trading preferences, their trading partners' debts' "safety", interest rate, size, and some randomness. See in more details below on how rankings and cakes are generated by each country at each round, and on how endowments change according to countries' behaviours.
    
#### Methodology

We present here the detailed code used for the simulations. For that we first calibrate the model to match the main trends and events observed from 1945 to now (2020). Being able to capture these features of past history will provide confidence and intuition in the model's inner workings. That lead us to provide several scenarii in how the models' parameters will change following Covid-19's increase in debts *(these parameters from the model being for instance the change in debt levels, while debt absorption speed are maintained roughly constant. Different degrees in shifts in trade and supply chain preferences are also considered)*. We thus generate 50 more rounds after a common departing point, and report here the different outcomes, and the different probabilities of each outcome (since they are intervention of random perturbations in the model the probabilities of each outcome is calculated based on how many of different random sequence generate this specific outcome).

#### Results

The simulations of extra Covid debts show that :

- in the case of everything constant (pre Trump US and China trade relations) debts are overall sustainable, with adjustements in reserve levels of countries depending on how affected by the crisis they were
- similar results in the case of everything constant, with just additional reliance on trade with China (for instance for masks, medical supplies etc). They are also small probabilities of increased shares of RMB within the international reserve repartition, but that comes with higher overall reserve levels for all 12 countries of the simulation

- in the case of everything constant among all countries except China and US, who in that scenario stop trading with each other, they are small probabilities of a global contraction (less reserves, less debts, less GDP growth) that preserves the overall shares of each country in the global economy - except for China in the global economy which suffers more. However in these small probability event the US go through an initial bubble fueled by the decrease of China, that then collapses back to the initial level (and note that after this collapse China's share is also back to the initial level - but with the cost of a global contraction, especially for all EM markets). **And with the caveat provided just above that this debt model doesn't take into account other incentives and consequences created by such dynamics - small probabilities in economic contraction in one country might leading to escalating tensions on other dimensions as well*

<img src="Screen%20Shot%202020-04-04%20at%209.50.20%20PM.png" /> 

- in the case of stopping trade between China and the US **AND** additional reliance by other countries on China, there is high probability for a sudden takeover of the role of the US as reserve currency China, accompanied by strong contraction for every other country. **Again, with the previous caveat of unaccounted destabilizing geopolitical consequences**

<img src="Screen%20Shot%202020-04-04%20at%209.57.55%20PM.png" /> 


- finally, in the case of our proposed stabilization scheme *(details in the paper - in the model it is just reflected as more willingness from participating countries to accept others' devaluation - see the commented code in the last cells of this notebook. Un-implemented here is also the extra information each country will have on how much latitude they will have at each round on their debt, which should also help in addition of the effects produced here to stabilize debts, interest rates and balance global reserve currencies)* the effects of all above scenarii are smoothen, with much more balanced reserve currencies (at least 3 currencies accounting each more than 20% of the global share of reserves, with the exact currencies - most often Germany ie the Euro, or the JPY - switching depending on the random shocks, but without adverse effects on all countries - including on the US even though the dollar loses its monopoly as reserve currency)


<img src="Screen%20Shot%202020-04-04%20at%2010.12.50%20PM.png" /> 

<img src="Screen%20Shot%202020-04-04%20at%2010.13.52%20PM.png" /> 



#### Caveat

Model incomplete. Among the many limitations of this model :

- no military investment taken into account
- no legacy beyond endowments

Since they are the game changerss, paper examines how this incentivizes military investment to change the situation.

*Note that here all commands are interpreted by Octave instead of Matlab as the code is originally intended to. There are only two differences - the digraph function to plot the trade preference among countries graph, and the sprandsym (random initialization of the trade preference graph) function which is slightly more elaborate in Matlab. Both original Matlab codes and Octave code for these two points are explicit here (cell and cell), with more details if you want to copy paste the code and run in Matlab insteaed of here.* 

*Help on commands is available using the `%help` magic or using `?` with a command.*

Please feel free to reach back to nxyzhang@mit.edu, and to play/expand the model at will !


In [ ]:
# Exogeneous parameters of the simulation to be played with 

number_of_countries = 12 %total number of countries in the simulation. Here we are running the simulations with the US, UK, France, Germany, Japan, China, plus 6 other randomcountries
number_of_rounds = 100 %total number of "cake eating rounds" (roughly equals to 1year in real time)

interest_rate = (0.03).*ones(number_of_countries,1); %initial interest rates - more below on how interest rates change according to how successfully debt is absorbeb at each round.
interest_rate_history = zeros(number_of_countries,number_of_rounds);

uni=0.7;%uni is used in the random generation of the trade preference matrix - a number_of_countries x number_of_countries matrix in which the number line i, column j indicates how interconnected the economies of countries i and j are. This random generation is done with the function R = sprandsym(n,density) - in our case n=number_of_countries and density = uni, which returns a symmetric random, n-by-n, sparse matrix with approximately density*n*n nonzeros; each entry is the sum of one or more normally distributed random samples, and (0 <= density <= 1) .For calibration uni=0.3 creates with good probability if there are 6 countries in total 3 trade coalitions, one with 3 countries, one with 2, and 1 in autarky.

%If we were in Matlab we would have added the parameter rc=0.02, still for this sprandsym function, which denotes how inequal trade between countries can be (the smaller the more). Indeed,  R = sprandsym(n,density,rc) returns a matrix with a reciprocal condition number equal to rc. The distribution of entries is nonuniform; it is roughly symmetric about 0; all are in [−1,1]. If rc is a vector of length n, then R has eigenvalues rc. Thus, if rc is a positive (nonnegative) vector then R is a positive (nonnegative) definite matrix. In either case, R is generated by random Jacobi rotations applied to a diagonal matrix with the given eigenvalues or condition number. 

In [ ]:
# We initialize here the recording vectors that will collect the history of the simulation, to plot the graphs at the end of this notebook

endowments_reserves_history = zeros(number_of_countries,number_of_rounds); %The records of how much cakes in total each country has accumulated at each round (see cell just below for the definition of a cake)
endowments_debts_history = zeros(number_of_countries,number_of_rounds); %The records of how much cakes (ie debts) in total each country has generated up to round t (see cell just below for the definition of a cake)
sum_cakes_history = zeros(1,number_of_rounds); %The records of each country's cake size for each round (see cell just below for the definition of a cake)
sum_of_generated_cakes = 0; %for cumulated

abso_speed_history = zeros(number_of_countries,number_of_rounds); %The records of the eating speed of each country for each round (see cell just below for the definition of the eating speed)
abso_history = zeros(1,number_of_rounds); 
sum_of_abso_speed = 0; %for cumulated

devaluation_history = zeros(number_of_countries,number_of_rounds); %The records of the cumulated number of times which each country devalued up to round i
ranking_history = zeros(number_of_countries,number_of_countries); %The records of the order in which each country ranked other countries' cake to be eaten at each round(see cell just below for the definition of a cake)
history_counter = ones(number_of_countries,1); %used in which round to determine sizes of generated cakes and cake eating speeds (see within round cell below)
deval_history = zeros(number_of_countries,1); %idem

duration_T_updated_history = zeros(1,number_of_rounds); %The records of how long was the timer for each round (see cell just below for the definition of a timer)
duration_T = 1; % There's a timer for each round. A round ends when this timer ends, or when countries ate all the cakes if faster than the timer (the timer changes at each round depending on the total cake sizes and total eating speed ratio, so that it increases rather steadily)
fraction_ranking = (1/3); % Used for calibration
duration_T_updated = duration_T;
successfully_loaded_cake_last_round = zeros(number_of_countries,1);
successfully_loaded_cake_last_round = zeros(number_of_countries,1);

sum_of_generated_cakes = 0;
sum_of_abso_speed = 0;

In [ ]:
# Now we set up initial conditions to roughly match that of 1945 : i.e. we define the initial endowments (debts owed by each country and detained by which countries) and the initial cake eating speed 

% We initialize so that the initial numbers here in the endowments matrix corresponds to the initial percentage (0.11 = 11%) of total world GDP. 
% The endowments matrix is read as reserves accumulated by a country i for lines i of the endowments matrix, and cakes generated by country j for the columns j of the endowments matrix. So the endowments matrix is really just keeping in time record of how much each country i is holding debts from each other country j.

endowments = [0.11 0.01 0.001 0.02 ;%0 0 0 0 0 0 0 0; %for the US, its debt in 1945 being of 300 Bdollar ie 43% of US GDP 1945 so 11% of world GDP then
    0 0.05 0.001 0.03 ;% 0 0 0 0 0 0 0 0;%for the UK - the UK debt of 1945 being of around 30B$ ie roughly 10% of US debt, of which 3B$ owed by US
    0 0 0.001 0.01 ] %0 0 0 0 0 0 0 0];%for France 
endowments = [endowments rand([3,8]).*0.001]; %random small intialization for the other countries
endowments = [endowments ; rand([9,number_of_countries]).*0.001] %random small intialization for the other countries
endowments=endowments.*300; %scaling up (see below - we want the values here to avoid being lower than 1 for them to not converge to 0)

initial_debts = sum(endowments,1)';

In [ ]:
# We now set up initial conditials for the cake eating speed of each country (for that we use as proxy their share in world GDP)

abso_speed = [0.25; 0.15; 0.1; 0.01; 0.05; 0.05 ]; %US 1945 GDP being around 25% of 1945 World GDP, UK 15%, France 10%, Germany 1%, Japan 5%, China 5%
abso_speed = [abso_speed  ; rand([6,1]).*0.01]; %random small initialization for the other countries
abso_speed = abso_speed.*300; %scaling up (see below - we want the values here to avoid being lower than 1 for them to not converge to 0)

abso_speed_init = abso_speed;
initial_pos = 100.*rand([number_of_countries,1]);
initial_pos_init =initial_pos;
initial_pos_round = initial_pos;
cakes_from_previous_round = initial_pos;
abso_speed_from_previous_round = abso_speed;

In [ ]:
# We initialize now the initial trading preferences of the countries. 

ranked_list = zeros(number_of_countries,number_of_countries);

%at first US, UK and France trading with each other in 1945 (until 1953 - for which see later sections of the code)
trading_preferences = [3 2 2 ; %US prefers doing business with others
    2 3 2 ; %UK prefers with itself
    2 2 3]; %France prefers with itself as well

trading_preferences = [trading_preferences ones(3,9)];

trading_preferences = [trading_preferences ; ones(9,12)];
for j=1:number_of_countries
    ranked_list(j,:) = randperm(number_of_countries);
    trading_preferences(j,j)=0;
    tag{j}=strcat('country ',num2str(j));
end
tag{1} = 'USA';
tag{2} = 'UK';
tag{3} = 'France';
tag{4} = 'Germany';
tag{5} = 'Japan';
tag{6} = 'China';


In [ ]:
# Now we go into rounds of cake eating, that are looped over. Everything important happens in this cell !


for k=1:number_of_countries
    %trading_preferences(k,k)=trading_preferences(k,k)+1;
    [~,trading_order]=sort(trading_preferences(k,:).*(ones(1,number_of_countries)+endowments(k,:)),'descend');
    %could rank by size of trade and by size of trading partner
    res=sum(trading_preferences(k,:)~=0);
    if res>0
        for l=1:res
            ressort_count=find(ranked_list(k,:)==trading_order(l));
            if ressort_count>l
                for m=1:(ressort_count-l)
                    ranked_list(k,ressort_count+1-m)=ranked_list(k,ressort_count-m);
                end
                ranked_list(k,l)=trading_order(l);
            else
            end
        end
    end
end
ranked_list_true_init = ranked_list;


for i=1:number_of_rounds
    

    %add Marshall %The largest recipient of Marshall Plan money was the United Kingdom (receiving about 26% of the total), followed by France (18%) and West Germany (11%)
    %The $17 billion was in the context of a US GDP of $258 billion in 1948,
    %that new plan gave away about $7.5 billion annually until 1961 when it was replaced by another program
    
    if i==8
        %initial_debts(4,1)=0;
        %initial_debts(5,1)=0;
        
        successfully_loaded_cake_last_round(4,1)=1;
        %history_counter(4,1)=history_counter(4,1)-deval_history(4,1);
        deval_history(4,1)=0;
        
        successfully_loaded_cake_last_round(5,1)=1;
        %history_counter(5,1)=history_counter(5,1)-deval_history(5,1);
        deval_history(5,1)=0;
        
        trading_preferences(1,:) = [ 2 2 2 3 3 1 1 1 1 1 1 1 ];
        trading_preferences(2,:) = [ 3 2 2 2 2 1 1 1 1 1 1 1 ];
        trading_preferences(3,:) = [ 3 2 2 2 2 1 1 1 1 1 1 1 ];
        trading_preferences(4,:) = [ 3 2 2 3 3 1 1 1 1 1 1 1 ];
        trading_preferences(5,:) = [ 3 3 2 3 3 1 1 1 1 1 1 1 ];
    end
    
    if (i>3 && i<7)
        %endowments(2,1)=endowments(2,1)+0.8*mean(mean(endowments));
        %endowments(3,1)=endowments(3,1)+0.6*mean(mean(endowments));
        endowments(4,1)=endowments(4,1)+0.4*mean(mean(endowments));
    end
    
    if (i>7 && i<10)
        endowments(5,1)=endowments(5,1)+0.8*mean(mean(endowments));
    end
    %add forgiving Germany if i==8 (1953) endowments 8 ==0 and trade. Same
    %with Japan
    
    %China opening FDI vs US GDP ?
    if (i>45 && i<75)
        endowments(6,1)=endowments(6,1)+(i./1000)*mean(mean(endowments));
    end
    if i==45
        successfully_loaded_cake_last_round(6,1)=1;
        %(6,1)=history_counter(6,1)-deval_history(6,1);
        deval_history(6,1)=0;
        trading_preferences(1,:) = [ 2 2 2 2 2 3 1 1 1 1 1 1 ];
        trading_preferences(2,:) = [ 1 1 1 1 2 3 1 1 1 1 1 1 ];
        trading_preferences(3,:) = [ 1 1 1 1 2 3 1 1 1 1 1 1 ];
        trading_preferences(4,:) = [ 2 2 2 3 2 3 1 1 1 1 1 1 ];
        trading_preferences(5,:) = [ 2 2 2 2 3 3 1 1 1 1 1 1 ];
        trading_preferences(6,:) = [ 3 3 3 3 3 3 2 2 2 2 2 2 ];
        for m=7:12
            trading_preferences(m,6) = 3;
        end
        
    end
    
    cakes = sqrt(sum(endowments,1)'.*(1+rand(number_of_countries,1)));
    abso_speed = sqrt((sum(endowments,2)+sum(endowments,1)').*(1+rand(number_of_countries,1)));
    
    sum_of_generated_cakes = sum_of_generated_cakes + cakes;
    sum_of_abso_speed = sum_of_abso_speed + abso_speed;
    
    initial_pos_round = cakes;    %duration_T_updated = (15+rand(1,1))*sum(cakes)./sum(abso_speed);
    duration_T_updated = 50.*(1+rand(1,1))*sum(cakes)./sum(abso_speed);
    
    cakes_from_previous_round = cakes;
    abso_speed_from_previous_round = abso_speed;
    
    abso_history(1,i)=sum(abso_speed);
    sum_cakes_history(1,i)=sum(cakes);
    
    time_counter=0;
    cakes_counter=0;
    rank_counter = ones(number_of_countries,1);
    removed_cake_row = zeros(number_of_countries,1);
    k=1;
    latest_finished_cake = 0;
    will_be_totally_consumed = zeros(number_of_countries,1);
    while (duration_T_updated>0 && cakes_counter<number_of_countries) %&& rank_counter_tot<number_of_countries
        initial_pos_updated = cakes;
        diff_between_wished_and_offered = zeros(number_of_countries,1);
        
        for l=1:number_of_countries
            if removed_cake_row(ranked_list(l,1))>0
                o=1;
                u=1;
                if  removed_cake_row(ranked_list(l,o+1),1)>0
                    u=o+1;
                    while ((u<(number_of_countries-1)) && (removed_cake_row(ranked_list(l,u+1),1)>0))
                        u=u+1;
                    end
                    ranked_list(l,o)=ranked_list(l,u+1);
                
                else
                    ranked_list(l,o)=ranked_list(l,o+1);
                    
                end
                rank_counter(j,1)=rank_counter(j,1)+1;
            end
        end
        
        for j=1:number_of_countries
            if removed_cake_row(j,1)>0
            else
                if sum(abso_speed(ranked_list(:,1)==j))>cakes(j,1)
                    will_be_totally_consumed(j,1)=will_be_totally_consumed(j,1)+1;
                    diff_between_wished_and_offered(j,1) = sum(abso_speed(ranked_list(:,1)==j))/initial_pos_updated(j,1);
                end
            end
        end
        
        if sum(diff_between_wished_and_offered)==0
            for j=1:number_of_countries
                if removed_cake_row(j,1)>0
                else
                    diff_between_wished_and_offered(j,1) = sum(abso_speed(ranked_list(:,1)==j))/initial_pos_updated(j,1);
                    
                    will_be_totally_consumed(j,1)=will_be_totally_consumed(j,1)+1;
                end
            end
        else
        end
        
        [~,consumption_order]=sort(diff_between_wished_and_offered,'descend');
        
        if sum(diff_between_wished_and_offered)==0
            [ii,jj]=find(~removed_cake_row);
            if size(ii)==1
                consumption_order(1)=ii;
            else
                for j=1:size(ii)
                    consumption_order(j)=ii(j);
                end
            end
        else
        end
        
        if (duration_T_updated-cakes(consumption_order(1),1))>0
            duration_T_updated = duration_T_updated - cakes(consumption_order(1),1);
            for j=1:number_of_countries
                if ranked_list(j,1)==consumption_order(1)
                    endowments(j,consumption_order(1)) = endowments(j,consumption_order(1))+(abso_speed(j,1)./(sum(abso_speed(ranked_list(:,1)==consumption_order(1)))))*initial_pos_updated(consumption_order(1),1);
                elseif cakes(ranked_list(j,1),1)==0
                    removed_cake_row(ranked_list(j,1),1)=removed_cake_row(ranked_list(j,1),1)+1;            
                else
                    endowments(j,ranked_list(j,1)) = endowments(j,ranked_list(j,1))+(abso_speed(j,1)*cakes(consumption_order(1),1))/(sum(abso_speed(ranked_list(:,1)==consumption_order(1))));
                    cakes(ranked_list(j,1),1)=initial_pos_updated(ranked_list(j,1),1)-(sum(abso_speed(ranked_list(:,1)==ranked_list(j,1)))*cakes(consumption_order(1),1))/(sum(abso_speed(ranked_list(:,1)==consumption_order(1))));
                end
            end
            time_counter = time_counter + cakes(consumption_order(1),1);
            cakes(consumption_order(1),1) = 0;
            removed_cake_row(consumption_order(1),1)=removed_cake_row(consumption_order(1),1)+1;
            
        else
            cakes(consumption_order(1),1) = cakes(consumption_order(1),1) - duration_T_updated;
            
            time_counter = time_counter + duration_T_updated;
            
            for j=1:number_of_countries
                if ranked_list(j,1)==consumption_order(1)
                    endowments(j,consumption_order(1)) = endowments(j,consumption_order(1))+(abso_speed(j,1)./(sum(abso_speed(ranked_list(:,1)==consumption_order(1)))))*duration_T_updated;
                    
                elseif cakes(ranked_list(j,1),1)==0
                    removed_cake_row(ranked_list(j,1),1)=removed_cake_row(ranked_list(j,1),1)+1;
                    
                else
                    endowments(j,ranked_list(j,1)) = endowments(j,ranked_list(j,1))+(abso_speed(j,1)*duration_T_updated)/(sum(abso_speed(ranked_list(:,1)==consumption_order(1))));
                    cakes(ranked_list(j,1),1)=cakes(ranked_list(j,1),1)-(abso_speed(j,1)*duration_T_updated)/(sum(abso_speed(ranked_list(:,1)==consumption_order(1))));
                end
            end
            
            duration_T_updated = 0;
            
        end
        latest_finished_cake = consumption_order(1);
        
        cakes_counter = cakes_counter + 1;
        k=k+1;
        duration_T_updated_history(1,i)=duration_T_updated;
    
    end
    
    for j=1:number_of_countries
        if (initial_pos_round(j,1)-cakes(j,1))>0
            endowments(:,j)=endowments(:,j)*(1-(cakes(j,1)/initial_pos_round(j,1)));   
        else
        end
        if cakes(j,1)==0
            history_counter(j,1) = history_counter(j,1) + 1;
            successfully_loaded_cake_last_round(j,1) = 1;
            interest_rate(j,1)=interest_rate(j,1)./(1+rand(1,1));
        else
            interest_rate(j,1)=min(1,max(interest_rate_history(j,1),interest_rate(j,1).*(1+rand(1,1))));
            deval_history(j,1) = deval_history(j,1)+1 ;
        end
    end
    [~,success_order]=sort(history_counter,'descend');
    ranked_list = repmat(success_order',number_of_countries,1);
    
    
    
    for k=1:number_of_countries
        
        r1 = randperm(number_of_countries,2*round(number_of_countries*fraction_ranking));
        
        
        for j=1:round(number_of_countries*fraction_ranking)
            inter=ranked_list(k,r1(1,j));
            ranked_list(k,r1(1,j))=ranked_list(k,r1(1,2*j));
            ranked_list(k,r1(1,2*j))=inter;
        end
    end
    
    for k=1:number_of_countries
        [~,trading_order]=sort(trading_preferences(k,:).*(ones(1,number_of_countries)+max(0,successfully_loaded_cake_last_round(k,1).*(history_counter(k,1)-deval_history(k,1)).*endowments(k,:)*(interest_rate(k,1)))),'descend');
        res=sum(trading_preferences(k,:)~=0);
        if res>0
            for l=1:res
                ressort_count=find(ranked_list(k,:)==trading_order(l));
                if ressort_count>l
                    for m=1:(ressort_count-l)
                        ranked_list(k,ressort_count+1-m)=ranked_list(k,ressort_count-m);
                    end
                    ranked_list(k,l)=trading_order(l);
                else
                end
            end
        end
        
        interest_rate_history(k,i) = interest_rate(k,1);
    end
    ranked_list_init = ranked_list;
    endowments_reserves_history(:,i) = sum(endowments,2);
    endowments_debts_history(:,i) = sum(endowments,1)';
    abso_speed_history(:,i) = abso_speed;
    devaluation_history(:,i) = deval_history;
    ranking_history(:,:,i) = ranked_list_init;
end


### Discussion on why we picked this scenario as the baseline calibration for testing the different covid one on top

So the graphs can either be all in one cell, or be plot individually one per cell. I'll let the reader copy paste the relevant pieces of code accordingly, that can be put in a following single cell as shown for example below.

*Also note that the digraph function used to plot the graph of the trade preferences among countries is not implemented yet in Octave. So the code is provided here but won't produce this specific graph.

In [ ]:
sum_ranking_history = zeros(number_of_countries,number_of_rounds);
i=1;
for j=1:number_of_countries
    for k=1:number_of_countries
        [~,indexes]=sort(ranking_history(k,:,i));
        score_for_one_country = ((number_of_countries+1)*ones(1,1)-indexes);
        sum_ranking_history(j,1) = sum_ranking_history(j,1)+score_for_one_country(1,j);
    end
end
sum_ranking_history_b = sum_ranking_history;

for i=2:number_of_rounds
    for j=1:number_of_countries
        for k=1:number_of_countries
            [~,indexes]=sort(ranking_history(k,:,i));
            score_for_one_country = ((number_of_countries+1)*ones(1,1)-indexes);
            sum_ranking_history(j,i) = sum_ranking_history(j,i)+score_for_one_country(1,j);
        end
    end
    sum_ranking_history(:,i)=(sum_ranking_history(:,i)+sum_ranking_history(:,i-1));%./(mean(sum_ranking_history(:,i-1)));
    sum_ranking_history_b(:,i)=sum_ranking_history(:,i)-sum_ranking_history(:,i-1);%./(mean(sum_ranking_history(:,i-1)));
end


figure;
A=trading_preferences;
G = digraph(A);
LWidths = 5*G.Edges.Weight/max(G.Edges.Weight);

subplot(2,3,1);
plot(G,'LineWidth',LWidths)
title('Trade preferences among the 12 (numbered) countries graph')


subplot(2,3,2);
plot(interest_rate_history')
title('Interest rate history')
legend(tag)
    
subplot(2,3,3);
plot(abso_speed_history')
title('Cake eating speed history')
%legend(tag)

subplot(2,3,4);
plot(sum_ranking_history_b')
title('Ranking history by country')
%legend(tag)

subplot(2,3,5);
plot(endowments_debts_history')
title('Cumulated debts generated by country history')
%legend(tag)

subplot(2,3,6);
plot(endowments_reserves_history')
title('Cumulated reserves by country history')
%legend(tag)

In [ ]:
figure;
plot(sum_ranking_history_b')
title('Ranking history by country')
legend(tag)

In [ ]:
figure;
plot(interest_rate_history')
title('Interest rate history')
legend(tag)

In [ ]:
figure;
plot(endowments_debts_history')
title('Cumulated debts generated by country history')
legend(tag)

In [ ]:
figure;
plot(abso_speed_history')
title('Cake eating speed history')
legend(tag)

In [ ]:
figure;
plot(interest_rate_history')
title('Interest rate history')
legend(tag)

In [ ]:
### Adding the Covid shock and then simulating 
Discussion on why we picked this scenario as the baseline calibration for testing the different covid one on top
